# A tutorial for deploying a model with Flask, Docker and Heroku

Deployment refers to the act of making your machine learning model available in a production environment, where it can be accessed and utilised by other tools, workflows and software. Deployment is typically one of the last stages in the machine learning workflow and can be one of the most difficult.

Flask is a web framework for Python, meaning that it provides functionality for building APIs and web applications. In this tutorial, we will explore:

1. using Flask to create a simple API to interface with a machine leanring model; and,
2. using Flask to create a simple web application that integrates our API with some basic html.

The aim of this tutorial is to introduce you to deploying machine learning models with Flask. I will not be giving an in-depth introduction to Flask here, I only intend to show how easy it is to deploy a model with Flask and to provide a foundation for which you can build off of to deploy your models in efficient and creative ways in the future.

We'll be using SageMaker’s implementation of the XGBoost algorithm to train and host a regression model to predict the age of abalone based on the classic abalone dataset hosted [here](https://archive.ics.uci.edu/ml/datasets/abalone). We aim to predict the age of abalone based on eight physical measurements. The data provided in the tutorial has been modified from the original, namely, the "Sex" characteristic has been one-hot-encoded and the "rings" target variable has had +1.5 added to it to represent the abalone age in years. The data is located in the [data folder](./data). It has been shuffled, one-hot-encoded and split into training and validation sets for you. The training set has 3341 rows and the validation set has 836 rows. The data looks like this:

<img src="docs/img/flask_images/fl_0.png" width="600">

If you're interested in other ways of deploying your machine learning model, check out [my other tutorial using Amazon SageMaker](deploy-with-sagemaker.ipynb).

# Contents

1. [Requirements](#1)
2. [Preparing the model we wish to deploy](#2)
3. [Setting up your directory structure and environment](#3)
4. [Model deployment](#4)
5. [Building and deploying a web API](#5)
6. [Building and deploying a web application](#6)
7. [End and next steps](#7)

# 1. Requirements <a id=1></a>

To complete this tutorial, you'll need the following:

- Heroku account. Register [here](https://www.heroku.com/).
- Heroku CLI. Download [here](https://devcenter.heroku.com/categories/command-line).
- The Postman app and a free account. Download/register [here](https://www.postman.com/).

# 2. Preparing the model we wish to deploy <a id=2></a>

In contrast to the [Amazon SageMaker tutorial](deploy-with-sagemaker.ipynb) where we trained and built a model using SageMaker, here we will develop the model we wish to deploy locally. The notebook [build_model.ipynb](flask/build_model.ipynb) in this repository builds a simple Random Forest regression model using the abalone dataset. To simplify things a little, this model is trained on only four input features: `['length', 'diameter', 'height', 'whole_weight']` as the four-feature model did not seem to be too much worse than the full-featured model for the purpose of this tutorial:

<img src="docs/img/flask_images/fl_1.png" width="600">

The [build_model.ipynb](flask/build_model.ipynb) notebook saves the trained model using `joblib` into the appropriate directory location (described in the next section).

# 3. Setting up your directory structure and environment <a id=3></a>

We'll be needing a specific directory structure to help us easily deploy our machine learning model. As I'll discuss in the next section, we have two options for deploying our model: 1. as a web api service; or, 2. as a web application. The directory structure (provided in this repository in the [deploy-with-flask directory](deploy-with-flask)) that we need to follow looks like this:

```shell
flask
├── build_model.ipynb  # this notebook contains the model building code
├── web_api
│   └── abalone_predictor.joblib  # this is the machine learning model we have built locally
│   └── app.py  # the file that defines our flask API
│   └── Procfile  # required by Heroku to help start flask app
│   └── requirements.txt  # file containing required packages
│   
└── web_application
    └── abalone_predictor.joblib  # this is the machine learning model we have built locally
    └── app.py  # the file that defines our flask API
    └── Procfile  # required by Heroku to help start flask app
    └── requirements.txt  # file containing required packages
    └── templates  # this subdirectory contains HTML templates to help us build the web application
    │   └── style.css  # css template to be used in web application
    └── static  # this subdirectory contains CSS style sheets
        └── home.html  # html template to be used in web application
        └── prediction.html  # html template to be used in web application
```

At this point, you should also set up your development environment. I've provided a [`requirements.txt`](deploy-with-flask/web_api/requirements.txt) file in the repository. I recommend creating a new virtual environment (I use conda, so: `$ conda create -n python=3.6`) and to then install the required packages from `requirements.txt` using `pip install -r requirements.txt`.

# 4. Model deployment <a id=4></a>

As mentioned previously, we have two options for deploying our abalone prediction model. We can:

1. develop a RESTful web API that accepts HTTP requests in the form of input data and returns a prediction;
2. build a web application with a HTML user-interface that interacts directly with our API.

We'll explore both options below.

# 5. Building and deploying a web API <a id=5></a>

It is extremely easy to create a RESTful API with Python and Flask. We already have the model we wish to deploy, we just need to create an API that allows users to access our model - by "access" I mean, we want users to be able to send data to our model and to receive a prediction in return.

## 5.1 Building the Flask API

All we need to create our API is a single Python file named `app.py`. This file is located in the [`web_api`](deploy-with-flask/web_api) folder in this repository. This tutorial is not a tutorial on how to use Flask, rather, I want to show you how you can easily deploy a model with the help of Flask. There are many good online resources for learning about Flask - as a starter, I highly recommend the free [Flask tutorial video series by Corey Schafer](https://www.youtube.com/playlist?list=PL-osiE80TeTs4UjLw5MM6OjgkjFeUxCYH). With that said, let's open up `app.py` and briefly discuss what's going on in the file.


<img src="docs/img/flask_images/fl_2.png" width="600">

The `app.py` module is extremely simple. Each section of the code is numbered and described below:

1. We first create an instance of the Flask class, every Flask application will have this line.
2. We then paste in a Python function that accepts as input our trained machine learning model and some input data and return the model prediction.
3. We then load up our pre-trained model.
4. The `@` symbol denotes a decorator. You don't need to know too much about decorators to understand what's going on here. Basically, we are defining our home page and populating it with some basic HTML text.
5. We the define a new route at the URL `/predict` which will accept json POST requests, make a prediction with our previosuly defined prediction function, and then return the result.
6. This piece of Python code simply allows us to start running our flask application by directly invoking the module with python from the command line - let's do that now!

## 5.2 Testing the Flask API

Open up a terminal and `cd` to the location of `app.py`. Then, type `python app.py`. You should see something like the following.

<img src="docs/img/flask_images/fl_3.png" width="600">

Copy-and-paste the URL `http://127.0.0.1:5000/` into your browser of choice (this is the IP address of your local machine followed by the port, 5000, that Flask runs on by default).

<img src="docs/img/flask_images/fl_4.png" width="600">

Great, our Flask app is up and running!

We can open up Postman to make sure that we can send JSON POST requests to our app and receive a prediction in return. To do that:

1. Open up Postman on your computer.
2. Click "Create a request".
3. Change the request to a "POST" request.
4. Enter the URL `http://127.0.0.1:5000/predict`.
5. Click the "Body" tab, click the "raw" radio button, and from the drop-down choose "JSON".
6. Paste the following into the body (feel free to change the numbers if you like):

```
{"length": 0.41,
"diameter": 0.33,
"height": 0.10,
"whole_weight": 0.36}
```

7. Click "Send". You should receive a prediction back (in my case, it was 9.14).

<img src="docs/img/flask_images/fl_5.png" width="600">

## 5.3 Deploying the API

Okay, so we have a working API, we now want to deploy it to the web so others can send requests. We will use Heroku to deploy our app but you could also use other services such as AWS.

1. Head over to [Heroku](https://dashboard.heroku.com/), log-in, and click "Create new app".
2. Choose a unique name for your app.

<img src="docs/img/flask_images/fl_6.png" width="600">

3. We will be using the Heroku CLI to deploy our model. All we have to do is follow the simple instructions provided (note that for more complex applications, you may choose to containerize everything in a Docker container to deploy to Heroku).

<img src="docs/img/flask_images/fl_7.png" width="600">

4. If you follow those commands, you should eventually see something like the following message verifying that your flask app has been deployed:

<img src="docs/img/flask_images/fl_8.png" width="600">

5. Your app is now live on the web and anyone can send API requests to it! Let's give it a try in Postman. Open up Postman and repeat the steps outlined above except now with the url `https://my-abalone-predictor.herokuapp.com/predict`. If you sent your request correctly, you should receive a model prediction in return. Awesome!

<img src="docs/img/flask_images/fl_9.png" width="600">

# 6. Building and deploying a web application <a id=6></a>

In section 5, we deployed our model as an endpoint that can receive JSON requests and return a prediction. Great! However, Flask has the ability to create entire web applications, not just a simple API, and I want to briefly introduce that functionality here. We only need to refactor our code a little bit and link it up with some html and css to create our web application.

We will use Flask to create a html form, accept data submitted to the form, and return a prediction using the submitted data. I won't go into too much detail here, I just want to show you what's possible and give you a foundation to build off. Let's open up our web application's [`app.py`](deploy-with-flask/web_application/app.py) file and go through the code step-by-step:

<img src="docs/img/flask_images/fl_10.png" width="600">

1. We'll be using `wtforms` and `flask_wtf` to help us build our form so we need to add those to our import list. We're also importing a few useful modules from `flask` itself to help us build our web app.
2. We create an instance of the Flask class and we also create a `SECRET_KEY` which basically allows us to store and use information specific to a user in a session (more on that [here](https://flask.palletsprojects.com/en/1.1.x/quickstart/#sessions)).
3. We again define our prediction function.
4. Load up our pre-trained model.
5. We now construct a simple form - there is an input for each of our input features, as well as a submit button.
6. We want our home page to actually return the form we just created. So we instantiate a form, we validate it (check that each field has some data), and we then redirect the user to a page "prediction" where results will be displayed. The home page will be rendered with the help of the `home.html` file located [here](deploy-with-flask/web_application/templates/home.html) and we are passing the `form` to the template so we can use it in the rendering of the page. 
7. The "prediction" page will store the input data as a dictionary and pass it to our model predict function. The page is rendered wit the help of the `prediction.html` file located [here](deploy-with-flask/web_application/templates/prediction.html), and uses the `results` output by our model prediction function.

With that done, let's test out our application.

## 6.2 Testing the web application

1. Open up a terminal and `cd` to the location of our web application's `app.py` file.
2. Type `python app.py` and then copy and paste the URL `http://127.0.0.1:5000/` into your browser. You should see something like the following.

<img src="docs/img/flask_images/fl_11.png" width="600">

3. Our web application is working! Let's try and make a prediction:

<img src="docs/img/flask_images/fl_12.png" width="600">

<img src="docs/img/flask_images/fl_13.png" width="600">

4. Looks like our predictions are working too!

This application is of course extremely simple and Flask is capable of building much more sophisticated web applications (have a look at the [docs](https://flask.palletsprojects.com/en/1.1.x/)), but hopefully this has given you a taste and some ideas as to what's possible with deploying your machine learning model as an application.

## 6.3 Deploying the web application

We now have a working application, let's deploy it to the web using Heroku.

1. Head over to [Heroku](https://dashboard.heroku.com/), log-in, and click "Create new app".
2. Choose a unique name for your app.

<img src="docs/img/flask_images/fl_14.png" width="600">

3. We will again be using the Heroku CLI to deploy our model. Once again, follow the simple instructions provided by Heroku to deploy your web application.

<img src="docs/img/flask_images/fl_15.png" width="600">

<img src="docs/img/flask_images/fl_16.png" width="600">

4. If you follow those commands, you should eventually see a message verifying that your web application has been deployed!
5. Open up the provided URL and share your application with the world!

# 7. End and next steps <a id=7></a>

Congratulations! You just deployed a model using Flask and Heroku. Hopefully this tutorial gave you some insight into how a machine leanring model can be deployed using these tools and how you might be able to expand upon the concepts presented to quickly and creatively deploy your models!

I recommend checking out the [Flask docs](https://flask.palletsprojects.com/en/1.1.x/) to learn more about Flask. There is also an excellent and thorough [Flask tutorial by Miguel Grinberg](https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world) that you might find useful.